# HW4 — Worked examples

This notebook demonstrates (Q1) Neural Cleanse *demo*, (Q2) Laplace mechanism examples, and (Q3) Fairness pipeline using `data.csv`. Run the cells in order.

## Q3 — Fairness (end-to-end demo)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from fairness import load_data, train_baseline_model, accuracy, disparate_impact, zemel_proxy_fairness, apply_promotion_demotion, retrain_with_swapped_labels

# load data
Xdf, y = load_data('data.csv')
sensitive = Xdf['gender'].to_numpy()
# numeric features only for the simple baseline
Xnum = Xdf.select_dtypes(include=[np.number]).drop(columns=['gender']).to_numpy()
Xtrain, Xtest, ytrain, ytest, sens_train, sens_test = train_test_split(Xnum, y.to_numpy(), sensitive, test_size=0.3, random_state=0)
clf = train_baseline_model(Xtrain, ytrain)
Xs_test = clf._scaler.transform(Xtest)
yproba = clf.predict_proba(Xs_test)[:,1]
ypred = (yproba >= 0.5).astype(int)
print('Baseline accuracy:', accuracy(ytest, ypred))
print('Baseline disparate impact:', disparate_impact(ytest, ypred, sens_test))
print('Baseline Zemel-proxy fairness:', zemel_proxy_fairness(Xs_test, ypred, sens_test))

In [ ]:
# Apply promotion/demotion mitigation
k = 10  # swap budget (example)
swap_mask = apply_promotion_demotion(Xs_test, yproba, ytest.to_numpy(), sens_test, k)
clf2 = retrain_with_swapped_labels(Xtrain, ytrain, swap_mask[:Xtrain.shape[0]] if swap_mask.size> Xtrain.shape[0] else swap_mask)
Xs_test2 = clf2._scaler.transform(Xtest)
yproba2 = clf2.predict_proba(Xs_test2)[:,1]
ypred2 = (yproba2 >= 0.5).astype(int)
print('Post-mitigation accuracy:', accuracy(ytest, ypred2))
print('Post-mitigation disparate impact:', disparate_impact(ytest, ypred2, sens_test))
print('Post-mitigation Zemel-proxy fairness:', zemel_proxy_fairness(Xs_test2, ypred2, sens_test))

## Q2 — Laplace mechanism examples

In [ ]:
from privacy import laplace_scale, add_laplace_noise, laplace_cdf_threshold, compose_epsilons
# Example: average income sensitivity and scale (fill with your problem numbers)
sens_avg = 200000/500.0  # example sensitivity for mean (replace with actual)
eps = 1.0
print('scale b for average:', laplace_scale(sens_avg, eps))
# probability example: P(noisy_response > threshold)
print('P(noisy > 505):', laplace_cdf_threshold(505, 500, sensitivity=1.0, epsilon=0.5))
print('Composition (0.5 + 0.5) ->', compose_epsilons([0.5, 0.5]))

## Q1 — Neural Cleanse (demo run)

In [ ]:
from neural_cleanse import load_model, reconstruct_trigger, detect_outlier_scales
import torch
from torch.utils.data import TensorDataset, DataLoader
# demo model + random data
model = load_model(demo=True, device='cpu')
X = torch.rand(200,1,28,28)
y = torch.randint(0,10,(200,))
ds = TensorDataset(X,y)
loader = DataLoader(ds, batch_size=32)
scales = []
for lbl in range(10):
    _,_,s = reconstruct_trigger(model, loader, lbl, device='cpu', steps=200, lr=0.2)
    scales.append(s)
print('scales:', scales)
print('detected attacked (demo):', detect_outlier_scales(scales))